In [1]:
# ✅ Install required packages
!pip install huggingface_hub
!pip install spacy
!pip install setfit['absa']
!pip install datasets

In [5]:
!spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
# ✅ Imports
from transformers import pipeline, EarlyStoppingCallback
from datasets import load_dataset, Dataset
from setfit import AbsaModel, AbsaTrainer, TrainingArguments

In [7]:
# ✅ Load ABSA-compatible model
model = AbsaModel.from_pretrained('BAAI/bge-small-en-v1.5')

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [41]:
from datasets import load_dataset

# Load dataset
data = load_dataset('tomaarsen/setfit-absa-semeval-restaurants', split='train')

# Remove 'conflict' examples
filtered_data = data.filter(lambda x: x['label'] != 'conflict')

# Map string labels to integer values
label2id = {"negative": 0, "neutral": 1, "positive": 2}

def convert_labels(example):
    return {
        "text": example["text"],
        "label": int(label2id[example["label"]])  # FORCE native int
    }

processed_data = filtered_data.map(convert_labels)

# Final type check (OPTIONAL DEBUG STEP)
print(type(processed_data[0]['label']))  # Should print: <class 'int'>


Map:   0%|          | 0/3602 [00:00<?, ? examples/s]

<class 'int'>


In [42]:
train = processed_data.select(range(20))
test = processed_data.select(range(20, 40))

In [43]:
train[0] # sample of data.

{'text': 'But the staff was so horrible to us.',
 'span': 'staff',
 'label': 0,
 'ordinal': 0}

In [38]:
# ✅ Training configuration
args = TrainingArguments(
    output_dir='model',
    num_epochs=4,
    batch_size=8,
    eval_strategy='steps',
    eval_steps=20,
    save_steps=20,
    load_best_model_at_end=True,
    metric_for_best_model='eval_aspect_embedding_loss',
    greater_is_better=False
)

In [44]:
# ✅ Define trainer
trainer = AbsaTrainer(
    model=model,
    args=args,
    train_dataset=train,
    eval_dataset=test,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

Map:   0%|          | 0/28 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [ ]:
trainer.train()  #4ed0da51cf184b1f89551e1596ec5c807cdd16d4 api key

In [ ]:
model.evaluate(test) #Testing the accuracy.

In [ ]:
model.predict("hii")#prompt